In [1]:
import requests, re, time
import torch, torchvision
from torch import nn, optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install mtcnn

     |████████████████████████████████| 2.3MB 5.8MB/s 


In [ ]:
import mtcnn
print(mtcnn.__version__)

0.1.0


In [ ]:
#The MTCNN takes as a input a single image and outputs the bounding box coordinates for that image where the face is.
#The code below takes in an input directory and runs the MTCNN on all the images in that directory and outputs in 
#output directory only the bounded images (only faces). 
#link to code below: https://stackoverflow.com/questions/65105644/how-to-face-extraction-from-images-in-a-folder-with-mtcnn-in-python

from mtcnn import MTCNN
import cv2
import os
def crop_image(source_dir, dest_dir, mode):
    if os.path.isdir(dest_dir)==False:
        os.mkdir(dest_dir)
    detector = MTCNN()
    source_list=os.listdir(source_dir)
    uncropped_file_list=[]
    for f in source_list:
        f_path=os.path.join(source_dir, f)
        dest_path=os.path.join(dest_dir,f)
        img=cv2.imread(f_path)
        try:
          data=detector.detect_faces(img)
        except:
          print(f_path)
          data==[]
          pass
        if data ==[]:
            uncropped_file_list.append(f_path)
        else:
            if mode==1:  #detect the box with the largest area
                for i, faces in enumerate(data): # iterate through all the faces found
                    box=faces['box']  # get the box for each face                
                    biggest=0                    
                    area = box[3]  * box[2]
                    if area>biggest:
                        biggest=area
                        bbox=box 
                bbox[0]= 0 if bbox[0]<0 else bbox[0]
                bbox[1]= 0 if bbox[1]<0 else bbox[1]
                img=img[bbox[1]: bbox[1]+bbox[3],bbox[0]: bbox[0]+ bbox[2]] 
                cv2.imwrite(dest_path, img)
            else:
                for i, faces in enumerate(data): # iterate through all the faces found
                    box=faces['box']
                    if box !=[]:
                        # return all faces found in the image
                        box[0]= 0 if box[0]<0 else box[0]
                        box[1]= 0 if box[1]<0 else box[1]
                        cropped_img=img[box[1]: box[1]+box[3],box[0]: box[0]+ box[2]]
                        fname=os.path.splitext(f)[0]
                        fext=os.path.splitext(f)[1]
                        fname=fname + str(i) + fext
                        save_path=os.path.join(dest_dir,fname )
                        cv2.imwrite(save_path, cropped_img)  
       
    return uncropped_file_list

In [ ]:
source_dir = '/content/drive/MyDrive/dataset/happy'
dest_dir = '/content/drive/MyDrive/dataset_faces_only/happy'
uncropped_files_list=crop_image(source_dir, dest_dir,1)

In [ ]:
source_dir = '/content/drive/MyDrive/dataset/sad'
dest_dir = '/content/drive/MyDrive/dataset_faces_only/sad'
uncropped_files_list=crop_image(source_dir, dest_dir,1)

In [ ]:
source_dir = '/content/drive/MyDrive/dataset/angry'
dest_dir = '/content/drive/MyDrive/dataset_faces_only/angry'
uncropped_files_list=crop_image(source_dir, dest_dir,1)

In [ ]:
source_dir = '/content/drive/MyDrive/dataset/neutral'
dest_dir = '/content/drive/MyDrive/dataset_faces_only/neutral'
uncropped_files_list=crop_image(source_dir, dest_dir,1)

In [198]:
xform = transforms.Compose([torchvision.transforms.Resize((224,224)),torchvision.transforms.RandomHorizontalFlip(),torchvision.transforms.RandomRotation(15), transforms.ToTensor()])
dataset_full = datasets.ImageFolder('/content/drive/MyDrive/dataset_faces_only', transform=xform)

In [199]:
dataset_full

Dataset ImageFolder
    Number of datapoints: 2085
    Root location: /content/drive/MyDrive/dataset_faces_only
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
           )

In [200]:
dataset_full.class_to_idx

{'angry': 0, 'happy': 1, 'neutral': 2, 'sad': 3}

In [201]:
n_all = len(dataset_full)
n_train = int(0.8 * n_all)
n_test = n_all - n_train
rng = torch.Generator().manual_seed(1549)
dataset_train, dataset_test = torch.utils.data.random_split(dataset_full, [n_train, n_test], rng)
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size = 4, shuffle=True)
loader_test = torch.utils.data.DataLoader(dataset_test, batch_size = 4, shuffle=True) #prepare dataset by splitting, same as usual

In [202]:
model = models.resnet18(pretrained=True)
model.avgpool.output_size=(None,None)
model.fc = torch.nn.Sequential(torch.nn.Linear(25088,2508),torch.nn.ReLU(),torch.nn.Linear(2508,250),torch.nn.ReLU(),torch.nn.Linear(250,4))

In [203]:
torch.cuda.device_count()

1

In [204]:
device = torch.device('cuda:0')
model = model.to(device)

In [205]:
criterion = nn.CrossEntropyLoss()

def run_test(model):
    nsamples_test = len(dataset_test)
    loss, correct = 0, 0
    model.eval()
    with torch.no_grad():
        for samples, labels in loader_test:
            samples = samples.to(device)
            labels = labels.to(device)
            outs = model(samples)
            loss += criterion(outs, labels)
            _, preds = torch.max(outs.detach(), 1)
            correct_mask = preds == labels
            correct += correct_mask.sum(0).item()
    return loss / nsamples_test, correct / nsamples_test

In [206]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [207]:
def run_train(model, opt, sched):
    nsamples_train = len(dataset_train)
    loss_sofar, correct_sofar = 0, 0
    model.train()
    with torch.enable_grad():
        for samples, labels in loader_train:
            samples = samples.to(device)
            labels = labels.to(device)
            opt.zero_grad()
            outs = model(samples)
            _, preds = torch.max(outs.detach(), 1)
            loss = criterion(outs, labels)
            loss.backward()
            opt.step()
            loss_sofar += loss.item() * samples.size(0)
            correct_sofar += torch.sum(preds == labels.detach())
    sched.step()
    return loss_sofar / nsamples_train, correct_sofar / nsamples_train

In [208]:
def run_all(model, optimizer, scheduler, n_epochs):
    for epoch in range(n_epochs):
        loss_train, acc_train = run_train(model, optimizer, scheduler)
        loss_test, acc_test = run_test(model)
        print(f"epoch {epoch}: train loss {loss_train:.4f} acc {acc_train:.4f}, test loss {loss_test:.4f} acc {acc_test:.4f}")

In [209]:
run_test(model)

(tensor(0.3508, device='cuda:0'), 0.22302158273381295)

In [210]:
run_all(model, optimizer, scheduler, 20)

epoch 0: train loss 1.0243 acc 0.5516, test loss 0.1909 acc 0.6763
epoch 1: train loss 0.6802 acc 0.7440, test loss 0.1680 acc 0.7338
epoch 2: train loss 0.5421 acc 0.8040, test loss 0.1460 acc 0.7770
epoch 3: train loss 0.4281 acc 0.8531, test loss 0.1706 acc 0.7434
epoch 4: train loss 0.3786 acc 0.8609, test loss 0.1952 acc 0.7386
epoch 5: train loss 0.2609 acc 0.9125, test loss 0.1046 acc 0.8345
epoch 6: train loss 0.1899 acc 0.9359, test loss 0.1051 acc 0.8417
epoch 7: train loss 0.1576 acc 0.9442, test loss 0.0967 acc 0.8585
epoch 8: train loss 0.1461 acc 0.9520, test loss 0.1112 acc 0.8225
epoch 9: train loss 0.1367 acc 0.9526, test loss 0.1037 acc 0.8489
epoch 10: train loss 0.1266 acc 0.9526, test loss 0.1117 acc 0.8297
epoch 11: train loss 0.1299 acc 0.9466, test loss 0.1160 acc 0.8345
epoch 12: train loss 0.1102 acc 0.9640, test loss 0.1109 acc 0.8513
epoch 13: train loss 0.1032 acc 0.9646, test loss 0.1146 acc 0.8345
epoch 14: train loss 0.1192 acc 0.9568, test loss 0.1108 a

In [ ]:
  torch.save(model,'/content/drive/MyDrive/Colab Notebooks/model.pt')